In [107]:
#Imports
import os
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.formula.api as sm
import numpy as np


In [108]:
#Set working directory
os.chdir('/Users/khangphamgia/Downloads')
#Load data
df_tax = pd.read_csv("/Users/khangphamgia/Downloads/Environmental_Taxes.csv")
#Drop multiple unncessary columns
df_tax = df_tax.drop(columns = ["ISO2", "ISO3", "Source", "CTS Code", "CTS Name", "CTS Full Descriptor", "1995", "1996", "1997", "1998", "1999","2000", "2020", "2021"])
df_tax.head()

,ObjectId,Country,Indicator,Unit,2001,2002,2003,2004,2005,2006,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,1,Albania,Environmental Taxes,Domestic Currency,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,4.399314e+10,4.781379e+10,4.754858e+10,5.114559e+10,5.341565e+10
1,2,Albania,Environmental Taxes,Percent of GDP,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.067206e+00,3.247163e+00,3.066373e+00,3.124865e+00,3.157133e+00
2,3,Albania,Taxes on Energy (including fuel for transport),Domestic Currency,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.774111e+10,4.094562e+10,4.040004e+10,4.352182e+10,4.516530e+10
3,4,Albania,Taxes on Energy (including fuel for transport),Percent of GDP,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2.631314e+00,2.780726e+00,2.605369e+00,2.659072e+00,2.669496e+00
4,5,Albania,Taxes on Pollution,Domestic Currency,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.782069e+09,1.879970e+09,1.941324e+09,2.226251e+09,2.625011e+09


In [109]:
#Search for unique countries in the dataset
df_tax['Country'].unique()
# Print out Vietnam rows
df_tax[df_tax['Country'] == 'Vietnam']

,ObjectId,Country,Indicator,Unit,2001,2002,2003,2004,2005,2006,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
1219,1314,Vietnam,Environmental Taxes,Domestic Currency,NaN,NaN,3.204000e+12,3.583000e+12,3.943000e+12,3.969000e+12,...,1.052100e+13,1.120100e+13,1.267600e+13,1.184900e+13,1.208700e+13,2.702000e+13,4.363200e+13,4.466600e+13,4.705000e+13,6.307460e+13
1220,1315,Vietnam,Environmental Taxes,Percent of GDP,NaN,NaN,5.200000e-01,5.000000e-01,4.300000e-01,3.700000e-01,...,3.800000e-01,3.200000e-01,3.100000e-01,2.700000e-01,2.500000e-01,5.200000e-01,7.700000e-01,7.100000e-01,6.700000e-01,8.200000e-01
1221,1316,Vietnam,Taxes on Energy (including fuel for transport),Domestic Currency,NaN,NaN,3.204000e+12,3.583000e+12,3.943000e+12,3.969000e+12,...,1.052100e+13,1.120100e+13,1.267600e+13,1.184900e+13,1.208700e+13,2.702000e+13,4.363200e+13,4.466600e+13,4.705000e+13,6.307460e+13
1222,1317,Vietnam,Taxes on Energy (including fuel for transport),Percent of GDP,NaN,NaN,5.200000e-01,5.000000e-01,4.300000e-01,3.700000e-01,...,3.800000e-01,3.200000e-01,3.100000e-01,2.700000e-01,2.500000e-01,5.200000e-01,7.700000e-01,7.100000e-01,6.700000e-01,8.200000e-01
1223,1318,Vietnam,Taxes on Pollution,Domestic Currency,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
1224,1319,Vietnam,Taxes on Pollution,Percent of GDP,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
1225,1320,Vietnam,Taxes on Resources,Domestic Currency,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
1226,1321,Vietnam,Taxes on Resources,Percent of GDP,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
1227,1322,Vietnam,Taxes on Transport (excluding fuel for transport),Domestic Currency,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
1228,1323,Vietnam,Taxes on Transport (excluding fuel for transport),Percent of GDP,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00


In [110]:
# Select Environmental Taxes
df_tax = df_tax[df_tax['Indicator'] == 'Environmental Taxes']
df_tax = df_tax.dropna()
df_tax["Country"].unique()

array(['Antigua and Barbuda', 'Argentina', 'Australia', 'Austria',
       'Bahamas, The', 'Bangladesh', 'Belgium', 'Belize', 'Bolivia',
       'Bulgaria', 'Cabo Verde', 'Cameroon', 'China, P.R.: Mainland',
       'Congo, Dem. Rep. of the', 'Congo, Rep. of', "Côte d'Ivoire",
       'Croatia, Rep. of', 'Cyprus', 'Czech Rep.', 'Denmark', 'Ecuador',
       'Estonia, Rep. of', 'Eswatini, Kingdom of', 'Finland', 'France',
       'Germany', 'Ghana', 'Greece', 'Guyana', 'Hungary', 'Iceland',
       'Ireland', 'Italy', 'Jamaica', 'Japan', 'Kenya', 'Kyrgyz Rep.',
       'Latvia', 'Lithuania', 'Luxembourg', 'Malaysia', 'Mali', 'Malta',
       'Mexico', 'Morocco', 'Netherlands, The', 'Nicaragua', 'Niger',
       'Norway', 'Panama', 'Papua New Guinea', 'Paraguay', 'Peru',
       'Poland, Rep. of', 'Portugal', 'Romania', 'Rwanda', 'Sierra Leone',
       'Slovak Rep.', 'Slovenia, Rep. of', 'South Africa', 'St. Lucia',
       'Sweden', 'Togo', 'Trinidad and Tobago', 'Tunisia',
       'Türkiye, Rep. of